## 1. Load essential libraries

In [1]:
import os, json
import pandas as pd
import numpy as np
import regex as re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag_sents
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## 2. Upload Data

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except:
    pass

Mounted at /content/drive


In [5]:
path = os.getcwd()+'/drive/MyDrive/Fiscal_Policy_Project/Data/Ministere_des_finances'

In [6]:
contents = [x for x in os.listdir(path) if not os.path.isfile(os.path.join(path,x))]

# List of all files in a list of tuples (folder, filename)
json_files = []
for x in contents:
    files = os.listdir(os.path.join(path, x))
    for file in files:
        if file.endswith('.json'):
            json_files.append((x, file))

# 3. Create Dataframe

In [7]:
#Create dataframe for files
df = pd.DataFrame(data=json_files, columns=["Filetype", "Filename"])

#get first and last files
df.head().append(df.tail())

<ipython-input-7-9f55ba16768a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.head().append(df.tail())


,Filetype,Filename
0,Speeches,Discours_2020-04-29_5856.json
1,Speeches,Discours_2016-12-13_6058.json
2,Speeches,Discours_2018-04-05_5998.json
3,Speeches,Discours_2010-06-24_201.json
4,Speeches,Discours_2011-02-08_366.json
1093,Speeches,Discours_2010-05-07_172.json
1094,Speeches,Discours_2019-01-28_5958.json
1095,Speeches,Discours_2011-01-17_501.json
1096,Speeches,Discours_2011-04-04_399.json
1097,Speeches,Discours_2019-11-08_5898.json


In [8]:
# Pandas dataframe that will have the contents of file DATE and TEXT along with Folder and name

jsons_data = pd.DataFrame(columns=['DATE', 'AUTHOR', 'TEXT', 'FOLDER', 'FILE NAME'])

for i, file in enumerate(json_files):
    with open(os.path.join(path, file[0], file[1]), 'r', encoding='utf-8') as fd:
      json_text = json.load(fd) 
      
      TEXT = json_text['text']
      DATE = json_text['published_date']
      AUTHOR = json_text['author']
        
      # Push a list of data into a pandas DataFrame at row given by 'index'
      jsons_data.loc[i] = [DATE, AUTHOR, TEXT, file[0], file[1]]
        
jsons_data

,DATE,AUTHOR,TEXT,FOLDER,FILE NAME
0,2020-04-29,Bruno Le Maire,\nSeul le prononcé fait foi \n1 \n \n \nMINIS...,Speeches,Discours_2020-04-29_5856.json
1,2016-12-13,None,Conférence numérique franco-allemande \nAccélé...,Speeches,Discours_2016-12-13_6058.json
2,2018-04-05,None,\n \nMINISTERE DE L'ECONOMIE \nET DES FINANCE...,Speeches,Discours_2018-04-05_5998.json
3,2010-06-24,Hervé Novelli,"Hervé NOVELLI, Secrétaire d’État chargé du Com...",Speeches,Discours_2010-06-24_201.json
4,2011-02-08,Pierre Lellouche,"Pierre LELLOUCHE, Secrétaire d’Etat chargé du ...",Speeches,Discours_2011-02-08_366.json
...,...,...,...,...,...
1093,2010-05-07,None,En conformité avec la Norme Spéciale de Diffus...,Speeches,Discours_2010-05-07_172.json
1094,2019-01-28,None,\nSeul le prononcé fait foi \n1 \n \n \nMINIS...,Speeches,Discours_2019-01-28_5958.json
1095,2011-01-17,Eric BESSON,"Monsieur le Président du directoire RTE, \nche...",Speeches,Discours_2011-01-17_501.json
1096,2011-04-04,Frédéric Lefebvre,"Frédéric LEFEBVRE,\nSecrétaire d’Etat chargé d...",Speeches,Discours_2011-04-04_399.json


In [18]:
text = jsons_data['TEXT'].values[0]
text

" \nSeul le prononcé fait foi \n1 \n \n \nMINISTERE DE L'ECONOMIE \nET DES FINANCES \n \n \n \n \n \n \n \n \nAudition par la commission des affaires économiques de l’Assemblée nationale  \nsur le plan de reprise de l’économie \n \n \nDiscours de Bruno Le Maire, \nministre de l'Economie et des Finances \n \n \n \nBercy (en visioconférence)  \n \nMercredi 29 avril 2020 \n \n \n \n \n \n \n \nContact presse :  \nCabinet de Bruno Le Maire \n \n \n01 53 18 41 13 \n \n \n \n\n \nSeul le prononcé fait foi \n2 \n \n \n \nMonsieur le Président de la commission des affaires économiques, \nCher Roland Lescure  \nMesdames et messieurs les Députés,  \n \nJe suis heureux de vous retrouver en vidéoconférence, et triste que cela ne soit pas directement \ndans la salle de la commission des affaires économiques, mais je pense que d’ici quelques \nsemaines, cela pourra être le cas. En tout cas, je crois qu’il était très important que nous puissions \néchanger entre nous au moment où la France connaît un

In [19]:
text = re.sub(r"\W"," ",text)
text

'  Seul le prononcé fait foi  1      MINISTERE DE L ECONOMIE  ET DES FINANCES                  Audition par la commission des affaires économiques de l Assemblée nationale   sur le plan de reprise de l économie      Discours de Bruno Le Maire   ministre de l Economie et des Finances        Bercy  en visioconférence      Mercredi 29 avril 2020                Contact presse     Cabinet de Bruno Le Maire      01 53 18 41 13           Seul le prononcé fait foi  2        Monsieur le Président de la commission des affaires économiques   Cher Roland Lescure   Mesdames et messieurs les Députés      Je suis heureux de vous retrouver en vidéoconférence  et triste que cela ne soit pas directement  dans la salle de la commission des affaires économiques  mais je pense que d ici quelques  semaines  cela pourra être le cas  En tout cas  je crois qu il était très important que nous puissions  échanger entre nous au moment où la France connaît une crise économique dont la gravité n a  aucun équivalent

In [21]:
text =re.sub(r"\d"," ",text)
text

'  Seul le prononcé fait foi         MINISTERE DE L ECONOMIE  ET DES FINANCES                  Audition par la commission des affaires économiques de l Assemblée nationale   sur le plan de reprise de l économie      Discours de Bruno Le Maire   ministre de l Economie et des Finances        Bercy  en visioconférence      Mercredi    avril                     Contact presse     Cabinet de Bruno Le Maire                               Seul le prononcé fait foi           Monsieur le Président de la commission des affaires économiques   Cher Roland Lescure   Mesdames et messieurs les Députés      Je suis heureux de vous retrouver en vidéoconférence  et triste que cela ne soit pas directement  dans la salle de la commission des affaires économiques  mais je pense que d ici quelques  semaines  cela pourra être le cas  En tout cas  je crois qu il était très important que nous puissions  échanger entre nous au moment où la France connaît une crise économique dont la gravité n a  aucun équivalent

In [23]:
text =re.sub(r"\s+[a-z]\s+"," ",text)
text

'  Seul le prononcé fait foi         MINISTERE DE L ECONOMIE  ET DES FINANCES                  Audition par la commission des affaires économiques de Assemblée nationale   sur le plan de reprise de économie      Discours de Bruno Le Maire   ministre de Economie et des Finances        Bercy  en visioconférence      Mercredi    avril                     Contact presse     Cabinet de Bruno Le Maire                               Seul le prononcé fait foi           Monsieur le Président de la commission des affaires économiques   Cher Roland Lescure   Mesdames et messieurs les Députés      Je suis heureux de vous retrouver en vidéoconférence  et triste que cela ne soit pas directement  dans la salle de la commission des affaires économiques  mais je pense que ici quelques  semaines  cela pourra être le cas  En tout cas  je crois qu il était très important que nous puissions  échanger entre nous au moment où la France connaît une crise économique dont la gravité a  aucun équivalent dans hist

In [27]:
re.sub(r"\s+"," ",text)

' Seul le prononcé fait foi MINISTERE DE L ECONOMIE ET DES FINANCES Audition par la commission des affaires économiques de Assemblée nationale sur le plan de reprise de économie Discours de Bruno Le Maire ministre de Economie et des Finances Bercy en visioconférence Mercredi avril Contact presse Cabinet de Bruno Le Maire Seul le prononcé fait foi Monsieur le Président de la commission des affaires économiques Cher Roland Lescure Mesdames et messieurs les Députés Je suis heureux de vous retrouver en vidéoconférence et triste que cela ne soit pas directement dans la salle de la commission des affaires économiques mais je pense que ici quelques semaines cela pourra être le cas En tout cas je crois qu il était très important que nous puissions échanger entre nous au moment où la France connaît une crise économique dont la gravité a aucun équivalent dans histoire contemporaine française aucun Cette crise se résume en un chiffre C est le niveau de la récession pour dans notre pays C est un c

# 4. Clean Data

In [ ]:
# Check for Na
jsons_data['AUTHOR'].isna().sum()

78

In [ ]:
def preprocess(txt):
    txt = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", txt)
    txt = txt.lower()
    txt = re.sub(r"\W"," ",txt) #supprimer les sauts de lignes
    txt = re.sub(r"\d"," ",txt) #supprimer les chiffres
    txt = re.sub(r"\s+[a-z]\s+"," ",txt)
    txt = re.sub(r"\s+[a-z]$"," ",txt)
    txt = re.sub(r"^[a-z]\s+"," ",txt)
    txt = re.sub(r"\s+"," ",txt) #enlever les espaces
    return txt


In [ ]:
jsons_data['Processed TEXT'] = jsons_data['TEXT'].apply(lambda txt: preprocess(txt))

In [ ]:
for index, row in jsons_data.iterrows():
    if row['Processed TEXT'] == '':
        jsons_data.drop(index, inplace=True)

In [ ]:
jsons_data['Processed TEXT']

0        seul le prononcé fait foi ministere de econom...
1       conférence numérique franco allemande accélére...
2        ministere de economie et des finances ème for...
3       hervé novelli secrétaire état chargé du commer...
4       pierre lellouche secrétaire etat chargé du com...
                              ...                        
1093    en conformité avec la norme spéciale de diffus...
1094     seul le prononcé fait foi ministere de econom...
1095    monsieur le président du directoire rte cher d...
1096    frédéric lefebvre secrétaire etat chargé de la...
1097     seul le prononcé fait foi ministere de econom...
Name: Processed TEXT, Length: 1098, dtype: object

# - 5. Remove Stopwords

In [ ]:
stopwords.words('french')

['au',
 'aux',
 'avec',
 'ce',
 'ces',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'et',
 'eux',
 'il',
 'ils',
 'je',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'ma',
 'mais',
 'me',
 'même',
 'mes',
 'moi',
 'mon',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 'sa',
 'se',
 'ses',
 'son',
 'sur',
 'ta',
 'te',
 'tes',
 'toi',
 'ton',
 'tu',
 'un',
 'une',
 'vos',
 'votre',
 'vous',
 'c',
 'd',
 'j',
 'l',
 'à',
 'm',
 'n',
 's',
 't',
 'y',
 'été',
 'étée',
 'étées',
 'étés',
 'étant',
 'étante',
 'étants',
 'étantes',
 'suis',
 'es',
 'est',
 'sommes',
 'êtes',
 'sont',
 'serai',
 'seras',
 'sera',
 'serons',
 'serez',
 'seront',
 'serais',
 'serait',
 'serions',
 'seriez',
 'seraient',
 'étais',
 'était',
 'étions',
 'étiez',
 'étaient',
 'fus',
 'fut',
 'fûmes',
 'fûtes',
 'furent',
 'sois',
 'soit',
 'soyons',
 'soyez',
 'soient',
 'fusse',
 'fusses',
 'fût',
 'fussions',
 'fussiez',
 'fussent',
 'ayant',
 'ayante',
 'ayantes',


In [ ]:
counter=0
def remove_stopwords(text,language,processed,verbose):
    word_list = text.split() 
    filtered_words = [word for word in word_list if word not in stopwords.words(language)]
    text = ' '.join(filtered_words)
    global counter
    counter+=1
    if verbose==1 and counter % 10 == 0:
        print(f"processed {counter}/{len(processed)}")
    if counter >= len(processed):
        counter=0
    return text

In [ ]:
jsons_data['Processed TEXT'] = jsons_data['Processed TEXT'].apply(lambda text:remove_stopwords(text,'french',jsons_data['Processed TEXT'],verbose=1))


processed 10/1098
processed 20/1098
processed 30/1098
processed 40/1098
processed 50/1098
processed 60/1098
processed 70/1098
processed 80/1098
processed 90/1098
processed 100/1098
processed 110/1098
processed 120/1098
processed 130/1098
processed 140/1098
processed 150/1098
processed 160/1098
processed 170/1098
processed 180/1098
processed 190/1098
processed 200/1098
processed 210/1098
processed 220/1098
processed 230/1098
processed 240/1098
processed 250/1098
processed 260/1098
processed 270/1098
processed 280/1098
processed 290/1098
processed 300/1098
processed 310/1098
processed 320/1098
processed 330/1098
processed 340/1098
processed 350/1098
processed 360/1098
processed 370/1098
processed 380/1098
processed 390/1098
processed 400/1098
processed 410/1098
processed 420/1098
processed 430/1098
processed 440/1098
processed 450/1098
processed 460/1098
processed 470/1098
processed 480/1098
processed 490/1098
processed 500/1098
processed 510/1098
processed 520/1098
processed 530/1098
pr

In [ ]:
with open(path+'/jsons_data_MF.pickle','wb') as f:
    pickle.dump(jsons_data,f)